In [2]:
import numpy as np
from scipy.special import erfc

def ewald_sum(positions, charges, box_length, alpha, k_max, real_space_cutoff):
    N = positions.shape[0]
    volume = box_length**3
    real_space_sum = 0.0
    reciprocal_space_sum = 0.0
    self_interaction = 0.0

    # Real space sum with cutoff
    for i in range(N):
        for j in range(i + 1, N):
            rij = positions[i] - positions[j]
            rij = rij - np.round(rij/box_length)*box_length
            r = np.linalg.norm(rij)
            if r < real_space_cutoff:
                real_space_sum += charges[i] * charges[j] * erfc(alpha * r) / r

    # Reciprocal space sum
    for h in range(-k_max, k_max + 1):
        for k in range(-k_max, k_max + 1):
            for l in range(-k_max, k_max + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                k_vector = 2 * np.pi * np.array([h, k, l]) / box_length
                k_sq = np.dot(k_vector, k_vector)        
                k_dot_r = np.dot(positions, k_vector)
                reciprocal_space_sum += np.sum(charges * np.cos(k_dot_r)) ** 2 * np.exp(-k_sq / (4 * alpha**2)) / k_sq

    reciprocal_space_sum *= 2 * np.pi / volume

    # Self-interaction correction
    for i in range(N):
        self_interaction -= (charges[i]**2 * alpha / np.sqrt(np.pi))

    # Total Ewald sum
    ewald_sum = real_space_sum + reciprocal_space_sum + self_interaction
    #print(f"Real: {real_space_sum}")
    #print(f"Reciprocal: {reciprocal_space_sum}")
    #print(f"Self: {self_interaction}")
    return ewald_sum


# Function to generate random positions and charges
def generate_random_system(N, box_length, seed):
    np.random.seed(seed)
    positions = np.random.uniform(0, box_length, (N, 3))
    charges = np.random.uniform(0, box_length, N)
    for i in range(0,N):
        charges[i] = 1.0 if i % 2 == 0 else -1.0

    return positions, charges



# NOTE
# Per alpha 3.5/(cell/2) stabile al variare del cutoff


# Example usage:
N = 500  # Number of particles
box_length = 10  # Length of the cubic box

# Maximum value of the reciprocal lattice vectors
real_space_cutoffs = 5 # Real space cutoff distance
alpha = 3.5/5  # Different Ewald parameters to test
seed = 42  # Seed for reproducibility

# Generate random positions and charges
positions, charges = generate_random_system(N, box_length, seed=seed)
k_max = [1,2,3,4,5,6,7,8,9,10,11,12,13]

print(f"Total charge:{np.sum(charges)}")

# Compute Ewald sum for different alpha values
for k in k_max:
    result = ewald_sum(positions, charges, box_length, alpha, k, real_space_cutoffs)
    print(f"{alpha}:{k}:{result}")

0.0
0.7:1:-188.79763707209372
0.7:2:-173.15851139042059
0.7:3:-168.7777627561514
0.7:4:-168.01704623018563
0.7:5:-167.95397813981702
0.7:6:-167.9474390056616
0.7:7:-167.94715074603712
0.7:8:-167.94714011337416
0.7:9:-167.9471397797128
0.7:10:-167.94713977345245
0.7:11:-167.9471397733826
0.7:12:-167.94713977338225
0.7:13:-167.94713977338225


In [26]:
import numpy as np

# Constants
L = 10.0  # Length of the cubic box
tolerance = 1e-3  # Convergence tolerance

# Particle properties (example for 2 particles)
q = np.array([1.0, -1.0])  # Charges
positions = np.array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]])  # Positions

def extended_coulomb_potential_energy(positions, charges, L, image_range):
    N = len(charges)
    energy = 0.0
    for i in range(N):
        for j in range(i + 1, N):  # Only consider each pair once
            for nx in range(-image_range, image_range + 1):
                for ny in range(-image_range, image_range + 1):
                    for nz in range(-image_range, image_range + 1):
                        displacement = positions[i] - (positions[j] + np.array([nx, ny, nz]) * L)
                        r_norm = np.linalg.norm(displacement)
                        if r_norm != 0:
                            energy += charges[i] * charges[j] / r_norm
    energy *= 1 / (4 * np.pi)
    return energy

# Ensure charge neutrality
total_charge = np.sum(q)
if total_charge != 0:
    raise ValueError("The system is not charge neutral. Total charge: {}".format(total_charge))

# Determine adequate image_range for convergence
prev_energy = None
image_range = 1

while True:
    total_energy = extended_coulomb_potential_energy(positions, q, L, image_range)
    
    if prev_energy is not None:
        relative_change = abs((total_energy - prev_energy) / prev_energy)
        print(f"Image range: {image_range} | Total Coulomb potential energy: {total_energy:.5e} J | Relative change: {relative_change}")
        if relative_change < tolerance:
            break
    
    prev_energy = total_energy
    image_range += 1

print(f"Converged energy: {total_energy:.5e} J with image range: {image_range}")

Image range: 2 | Total Coulomb potential energy: -5.01011e-01 J | Relative change: 1.5309276910952068
Image range: 3 | Total Coulomb potential energy: -9.55576e-01 J | Relative change: 0.9072957935539979
Image range: 4 | Total Coulomb potential energy: -1.56166e+00 J | Relative change: 0.6342595448232885
Image range: 5 | Total Coulomb potential energy: -2.31926e+00 J | Relative change: 0.4851268482846826
Image range: 6 | Total Coulomb potential energy: -3.22839e+00 J | Relative change: 0.39198799312184424
Image range: 7 | Total Coulomb potential energy: -4.28903e+00 J | Relative change: 0.3285367538096799
Image range: 8 | Total Coulomb potential energy: -5.50119e+00 J | Relative change: 0.28261960505159794
Image range: 9 | Total Coulomb potential energy: -6.86488e+00 J | Relative change: 0.2478888012111626
Image range: 10 | Total Coulomb potential energy: -8.38008e+00 J | Relative change: 0.22071837771601788
Image range: 11 | Total Coulomb potential energy: -1.00468e+01 J | Relative ch

KeyboardInterrupt: 